In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from os import path
from pathlib import Path  

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [8]:
# import files
data_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/data_matrix.csv') # Data matrix
gene_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Homo_sapiens.tsv') # Gene Symbol and ID
pathway_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Pathway_Down.csv') # all the pathways used in the heatmap
gene_to_pathway_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Gene_to_Pathway.csv') # every pathway and the relevant genes

# dataframe of metadata
data = pd.read_csv(data_file)
gene_info = pd.read_csv(gene_file, sep="\t")
pathway = pd.read_csv(pathway_file,index_col=0).fillna("0")
gene_to_pathway = pd.read_csv(gene_to_pathway_file)

In [9]:
pathways = pathway.index.tolist()
pathway_name = []
pathway_number = []
for i in range(len(pathways)):
    temp = pathway.iloc[i]
    temp2 = []
    for j in range(4):
        if temp[j] != "0":
            pathway_name.append(pathways[i])
            pathway_number.append(temp[j])
pathway_number = set(pathway_number)            

In [ ]:
# get gene IDs
# note that 7 percent (~252/3418 genes does not have matching symbol=ID)
Gene_IDs = []
for i in range(len(data.index.tolist())):
    Gene_name = data.iloc[i][0]
    
    # get gene symbol
    if "," in str(Gene_name):
        temp = str(Gene_name).split(",")
        if temp[0] != "":
            Gene_name = temp[0]
        else:
            Gene_name = temp[1].lstrip()
    
    # gene_symbol to ID
    ID = gene_info.loc[gene_info['Symbol'] == Gene_name]['GeneID'].tolist()
    if ID == []:
        Gene_IDs.append("None")
    else:
        Gene_IDs.append(ID[0])
    
    if (i%100) == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200


In [ ]:
# add the IDs column in and drop the genes that doesn't have an ID
data["ID"] = Gene_IDs
data = data[data['ID'] != "None"]

In [ ]:
# get the pathway
# only 1402/3166 genes belong to a functional pathway
pathways_by_gene = []
pathways_by_gene_column = []
for i in range(len(data.index.tolist())):
    temp_ID = data.iloc[i][28]
    
    # get corresponding pathways
    temp_pathways = gene_to_pathway[gene_to_pathway['Gene']== data.iloc[i][28]]["Pathway"].tolist()
    
    if len(temp_pathways) == 0:
        pathways_by_gene_column.append("none")
    else:
        pathways_by_gene.append(temp_pathways)
        pathways_by_gene_column.append(" ".join(temp_pathways))

In [ ]:
data["All Pathways"] = pathways_by_gene_column
data = data[data['All Pathways'] != "none"]

## output the data file as long as the gene is involved with a pathways ##

In [ ]:
filepath = Path('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Data_All_Pathways.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data.to_csv(filepath)  

## continue to shrink data only if a gene is involved with a pathway actually used in the heatmap ##

In [ ]:
# find the pathway that's involved with the heat map
pathways_involved = []
pathways_involved_column = []
for i in range(len(data.index.tolist())):
    a = pathways_by_gene[i]
    temp = []

    for j in a:
        if j in pathway_number:
            temp.append(j)
    
    if len(temp) == 0:
        pathways_involved_column.append("none")
    else:
        pathways_involved.append(temp)
        pathways_involved_column.append(" ".join(temp))

In [ ]:
data["Involved Pathways"] = pathways_involved_column
data = data[data['Involved Pathways'] != "none"]

In [ ]:
drugs = data.columns.values.tolist()[1:28]
functions = pathway.index.tolist()

In [ ]:
# create the finalmatrix
final_matrix = [[0.0] * len(drugs)]*len(functions)
counter=0
for i in range(len(data.index.tolist())):
    temp_pathways = pathways_involved[i]
    temp_values = data.iloc[i].tolist()[1:28]
    temp_functions = []
    for j in temp_pathways:
        a = pathway[pathway["pathway_1"] == j].index.tolist()
        if len(a)>0:
            temp_functions.append(a[0])
        else:
            a = pathway[pathway["pathway_2"] == j].index.tolist()
            if len(a)>0:
                temp_functions.append(a[0])
            else:
                a = pathway[pathway["pathway_3"] == j].index.tolist()
                if len(a)>0:
                    temp_functions.append(a[0])
                else:
                    a = pathway[pathway["pathway_4"] == j].index.tolist()
                    if len(a)>0:
                        temp_functions.append(a[0])
    for k in temp_functions:
        counter = counter + 1
        temp_index = functions.index(k)
        final_matrix[temp_index] = [final_matrix[temp_index][n] + temp_values[n] for n in range(len(temp_values))] 


In [ ]:
final_data_functions_by_drugs = pd.DataFrame(final_matrix,index = functions, columns=drugs)
final_data_functions_by_drugs

In [ ]:
filepath = Path('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Final_data_functions_by_drugs.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_data_functions_by_drugs.to_csv(filepath)  

## establish matrix as pathway by drugs ##